# graph_3_reg_20_30_w_01

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('../'))
sys.path.insert(0, os.path.abspath('../../'))
sys.path.insert(0, os.path.abspath('/home/hm-tlacherm/qlm_notebooks/notebooks_1.2.1/notebooks/master_thesis_qaoa/'))
sys.path.insert(0, os.path.abspath('/home/hm-tlacherm/qlm_notebooks/notebooks_1.2.1/notebooks/master_thesis_qaoa/ibm/'))
sys.path.insert(0, os.path.abspath('/home/hm-tlacherm/qlm_notebooks/notebooks_1.2.1/notebooks/master_thesis_qaoa/ibm/landscape/'))
sys.path.insert(0, os.path.abspath('/home/hm-tlacherm/qlm_notebooks/notebooks_1.2.1/notebooks/master_thesis_qaoa/ibm/landscape/simulator/'))

In [ ]:
import numpy as np
import os.path

import qiskit
provider = qiskit.IBMQ.load_account()
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import QAOA
from shared.QiskitMaxcut import *
from ibm.ibm_parameters import *

from matplotlib import pyplot as plt
%matplotlib inline

from max_cut_helper import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# ---- Define graph and MaxCut ----
filename = "graph_3_reg_20_30_w_01.txt"
graph = load_nx_graph_from(f"../data/{filename}")
opt_max_cut = -get_max_cut(graph.name)
max_cut = Maxcut(graph, opt_max_cut)
max_cut_qubo = max_cut.to_qubo()
max_cut.draw()

# GoemansWilliamson Solution

In [ ]:
from qiskit_optimization.algorithms import GoemansWilliamsonOptimizer

In [ ]:
optimizer = GoemansWilliamsonOptimizer(5)
result = optimizer.solve(max_cut_qubo)
mean, distribution = max_cut.analyse(result, print_output=True)
max_cut.plot_histogram(distribution, mean)

# Display Energy Landscape for p = 1

In [ ]:
#from landscape_helper import *

In [ ]:
#landscape_results_filename = f"landscape_simulator_{graph.name}_results.npy"

In [ ]:
#landscape = load_landscape_data(landscape_results_filename, max_cut)

In [ ]:
#min_beta, min_gamma, min_exp = describe_landscape(landscape)

In [ ]:
#from plot_helper import *

In [ ]:
# Plot landscape in 3D 
#plot_landscape_3d(landscape)

In [ ]:
# Plot Heatmap 
#heatmap = plot_heatmap(landscape)
#heatmap = display_minimum(heatmap, min_gamma, min_beta, min_exp)

#### Find initial points with TQA

In [ ]:
from tqa import calculate_tqa

In [ ]:
p = 1

In [ ]:
tqa_initial_points = calculate_tqa(graph, p)

#### Run QAOA with init_points, different optimizers and plot results 

In [ ]:
#from qaoa_helper import *

In [ ]:
#qnSpsa = QNSPSA(calculate_fidelity(create_qaoa(), max_cut_qubo), maxiter=50)
#optimizers = [
#    ('COBYLA',COBYLA(maxiter=50)),
#    ('SPSA', SPSA(maxiter=50)),
#    ('QNSPSA', qnSpsa),
#]

In [ ]:
#for optimizer_tuple in optimizers:
#    optimizer_name, optimizer = optimizer_tuple
#    # create qaoa 
#    qaoa = create_qaoa(optimizer = optimizer,
#            reps=p,
#            initial_point=tqa_initial_points,
#            with_callback=True
#           )
#    # run qaoa 
#    result, optimal_parameters, optimizer_history = run_qaoa_with_callback(qaoa, max_cut_qubo)
    
    # analyse results 
#    print(f"{optimizer_name} Optimizer")
#    mean, distribution = max_cut.analyse(result, print_output=True)
#    max_cut.plot_histogram(distribution, mean)
#    optimizer_history[2][-1] = mean
#    print()
    
    # Display Optimizer Results
#    counts, energy_values, maxcut_values, optimizer_gammas, optimizer_betas = optimizer_history
#    display_optimizer_path(heatmap, optimizer_gammas, optimizer_betas, maxcut_values, optimizer_name)
    
    # Plot Optimizer History MaxCut Evaluation # Values from landscape -> real values may deviate 
#    plot_optimizer_maxcut_history(counts, maxcut_values, optimizer_name)
    
    # Plot Optimizer History Energy Evaluation -> not MaxCutMean! 
#    plot_optimizer_energy_history(counts, energy_values, optimizer_name)
    

# Comparison

## Parameters

In [ ]:
eval_num = 50
max_p = 10
all_results = {}

## QAOA

In [ ]:
from qaoa_helper import *

In [ ]:
qaoa_p_means = []
qaoa_p_ratios = []
qaoa_p_approx_ratios = []

for p in range(1,max_p+1):
    m, r, ar = start_qaoa_evaluation(max_cut, eval_num=eval_num, reps=p)
    qaoa_p_means.append(m)
    qaoa_p_ratios.append(r)
    qaoa_p_approx_ratios.append(ar)

all_results["QAOA"] = [qaoa_p_means, qaoa_p_ratios, qaoa_p_approx_ratios]

In [ ]:
display_boxplots_results(qaoa_p_means, qaoa_p_ratios, qaoa_p_approx_ratios)

## QAOA with TQA

In [ ]:
from qaoa_helper import *

In [ ]:
tqa_p_means = []
tqa_p_ratios = []
tqa_p_approx_ratios = []
tqa_init_points = []

for p in range(1,max_p+1):
    tqa_initial_points = calculate_tqa(graph, p)
    m, r, ar = start_qaoa_evaluation(max_cut, eval_num=eval_num, reps=p, init_points=tqa_initial_points)
    tqa_p_means.append(m)
    tqa_p_ratios.append(r)
    tqa_p_approx_ratios.append(ar)
    tqa_init_points.append(tqa_initial_points)
    
all_results["TQA QAOA"] = [tqa_p_means, tqa_p_ratios, tqa_p_approx_ratios, tqa_init_points]

In [ ]:
display_boxplots_results(tqa_p_means, tqa_p_ratios, tqa_p_approx_ratios, prefix='TQA ')

## WarmStart QAOA

In [ ]:
from warmstart_helper import *

In [ ]:
ws_p_means = []
ws_p_ratios = []
ws_p_approx_ratios = []
ws_opt_epsilons = []

for p in range(1,max_p+1):
    opt_epsilon = optimize_epsilon(max_cut, reps=p)
    m, r, ar = start_ws_qaoa_evaluation(max_cut, eval_num=eval_num, reps=p, epsilon=opt_epsilon)
    ws_p_means.append(m)
    ws_p_ratios.append(r)
    ws_p_approx_ratios.append(ar)
    ws_opt_epsilons.append(opt_epsilon)
    
all_results["WarmStart QAOA"] = [ws_p_means, ws_p_ratios, ws_p_approx_ratios, ws_opt_epsilons]

In [ ]:
display_boxplots_results(ws_p_means, ws_p_ratios, ws_p_approx_ratios, prefix='WarmStart ')

### Recursive QAOA

In [ ]:
from recursive_qaoa_helper import *

In [ ]:
recursive_p_means = []
recursive_p_ratios = []
recursive_p_approx_ratios = []

for p in range(1,max_p+1):
    m, r, ar = start_recursive_evaluation(max_cut, eval_num=eval_num, reps=p)
    recursive_p_means.append(m)
    recursive_p_ratios.append(r)
    recursive_p_approx_ratios.append(ar)

all_results["Recursive QAOA"] = [recursive_p_means, recursive_p_ratios, recursive_p_approx_ratios]

In [ ]:
display_boxplots_results(recursive_p_means, recursive_p_ratios, recursive_p_approx_ratios, prefix='Recursive ')

### Recursive WarmStart QAOA

In [ ]:
from recursive_ws_helper import *

In [ ]:
ws_recursive_p_means = []
ws_recursive_p_ratios = []
ws_recursive_p_approx_ratios = []

for p in range(1,max_p+1):
    opt_epsilon = ws_opt_epsilons[p-1]
    m, r, ar = start_recursive_ws_qaoa_evaluation(max_cut, eval_num=eval_num, reps=p, epsilon=opt_epsilon)
    ws_recursive_p_means.append(m)
    ws_recursive_p_ratios.append(r)
    ws_recursive_p_approx_ratios.append(ar)
    
all_results["Recursive WarmStart QAOA"] = [ws_recursive_p_means, ws_recursive_p_ratios, ws_recursive_p_approx_ratios, ws_opt_epsilons]

In [ ]:
display_boxplots_results(ws_recursive_p_means, ws_recursive_p_ratios, ws_recursive_p_approx_ratios, prefix='Recursive WarmStart ')

## Results

In [ ]:
# Save results 
with open(f'comparison_simulator_{graph.name}_results.npy', 'wb') as f:
    np.save(f, all_results)

In [ ]:
from results_helper import *

In [ ]:
means_df, ratio_df, approx_ratios_df = generate_dataframes(all_results)

### Metric Mean

In [ ]:
means_df

### Metric Ratio 

In [ ]:
ratio_df

### Metric Approximation Ratio

In [ ]:
approx_ratios_df